In [97]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# 1) Load PyBaMM parameters and grab OCP functions
param = pybamm.ParameterValues("Chen2020")          # "Chen2020" or "Marquis2019", "Mohtat2020"
ocv_anode   = param["Negative electrode OCP [V]"]  # Graphite
ocv_cathode = param["Positive electrode OCP [V]"]  # NCM811

# 2) Build a fine SOC grid and compute OCV and differential capacity dQ/dV
soc_grid = np.linspace(0.01, 0.99, 500)

# Anode (graphite)
V_an = ocv_anode(soc_grid)
dVdQ_an = np.gradient(V_an, soc_grid)
dQdV_an = 1.0 / dVdQ_an

# Cathode (NCM811)
V_ca = ocv_cathode(soc_grid)
dVdQ_ca = np.gradient(V_ca, soc_grid)
dQdV_ca = 1.0 / dVdQ_ca

# 3) Interactive update function
def update(soc):
    soc_an = soc
    soc_ca = 1 - soc   # cathode depletes


    # Ensure SOC is within bounds
    # Anode
    soc_an = np.clip(soc_grid, 0.0, soc)
    V_an_cur = ocv_anode(soc_an)
    dQdV_an_cur = np.interp(soc_an, soc_grid, dQdV_an)
    #Cathode
    soc_ca = np.clip(soc_grid, 0.0, 1-soc)
    V_ca_cur = ocv_cathode(soc_ca)
    dQdV_ca_cur = np.interp(soc_ca, soc_grid, dQdV_ca)
    # Interpolate current differential capacity and voltage

    # Plot
    fig, ax = plt.subplots(figsize=(6, 5))

    # Plot differential capacity curves    
    battery_color={"gray": "#333333",
                       "bright_gray": "#E7E6E6",
                        "gray_blue": "#44546A",
                        "navy_blue": "#003B73",
                        "light_blue":"#5DADE2",
                        "orange": "#F39C12",
                        "green": "#70AD47"}
    ax.plot(dQdV_an, -V_an, label="Graphite tank", color=battery_color["gray"])
    ax.plot(dQdV_ca, -V_ca, label="NCM tank", color=battery_color["light_blue"])
    ax.plot(-dQdV_an, -V_an, color=battery_color["gray"])
    ax.plot(-dQdV_ca, -V_ca, color=battery_color["light_blue"])

    # Fill according to SOC
    ax.fill_betweenx(
        -V_an_cur, 
        dQdV_an_cur,
        0,
        color=battery_color["navy_blue"], alpha=0.8,
    )
    ax.fill_betweenx(
        -V_an_cur, 
        -dQdV_an_cur,
        0,
        color=battery_color["navy_blue"], alpha=0.8,
    )
    ax.fill_betweenx(
        -V_ca_cur, 
        -dQdV_ca_cur,
        0,
        color=battery_color["navy_blue"], alpha=0.8,
    )
    ax.fill_betweenx(
        -V_ca_cur, 
        dQdV_ca_cur,
        0,
        color=battery_color["navy_blue"], alpha=0.8,
    )

    # Add voltage difference arrow between anode and cathode at x=0
    ax.annotate(
        '',
        xy=(0, -V_ca_cur.min()), 
        xytext=(0, -V_an_cur.min()),
        arrowprops=dict(arrowstyle='<->', color='orange', linewidth=2)
    )
    mid_y = 0.5*( -V_ca_cur.min() -V_an_cur.min() )
    delta_V = V_ca_cur.min() - V_an_cur.min()


    ax.text(0.1, mid_y, f'ΔV = {delta_V:.3f} V', color='orange', va='center')
    # Labels and legend
    ax.set_xlabel("dQ/dV [Ah/V]")
    ax.set_ylabel("Voltage [V]")
    ax.set_title("Voltage vs Differential Capacity")
    ax.legend(loc="right")
    ax.grid(True)
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 0)
    yticks = np.arange(-5, 1, 1)   # e.g. [-5, -4, -3, -2, -1, 0]
    ax.set_yticks(yticks)

    # 2) Label them with their absolute values
    ax.set_yticklabels([abs(int(t)) for t in yticks])

    # 3) Move legend to the left inside the axes
    ax.legend(loc="center left")
    #ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{abs(x):.2f}"))
    plt.tight_layout()
    plt.show()

# 4) Create slider and display widget
soc_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01, description="SOC"
)
widgets.interact(update, soc=soc_slider)


interactive(children=(FloatSlider(value=0.5, description='SOC', max=1.0, step=0.01), Output()), _dom_classes=(…

<function __main__.update(soc)>